# Курс Введение в обработку естественного языка / Практическое задание урока 5. Part-of-Speech разметка, NER, извлечение отношений

## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации  
2. написать свой теггер как на занятии, но улучшить попробовать разные векторайзеры, добавить знание не только букв и слов но и совместно объединить эти признаки 
3. сравнить все реализованные методы сделать выводы

### загрузка данных

In [31]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyconll

In [ ]:
!mkdir datasets

In [ ]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [ ]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [ ]:
import nltk

from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

In [ ]:
def conver_to_test_data(full):
    res = []
    for sent in full:
        sub_res = []
        for token in sent:
            sub_res.append((token.form, token.upos))
        res.append(sub_res)
    return res

In [ ]:
def conver_to_test_sent(full):
    res = []
    for sent in full:
        sub_res = []
        for token in sent:
            sub_res.append(token.form)
        res.append(sub_res)
    return res

In [ ]:
train_data = conver_to_test_data(full_train)

In [ ]:
test_data = conver_to_test_data(full_test)
print(test_data[0:10])

[[('Алгоритм', 'NOUN'), (',', 'PUNCT'), ('от', 'ADP'), ('имени', 'NOUN'), ('учёного', 'NOUN'), ('аль', 'PART'), ('-', 'PUNCT'), ('Хорезми', 'PROPN'), (',', 'PUNCT'), ('-', 'PUNCT'), ('точный', 'ADJ'), ('набор', 'NOUN'), ('инструкций', 'NOUN'), (',', 'PUNCT'), ('описывающих', 'VERB'), ('порядок', 'NOUN'), ('действий', 'NOUN'), ('исполнителя', 'NOUN'), ('для', 'ADP'), ('достижения', 'NOUN'), ('результата', 'NOUN'), ('решения', 'NOUN'), ('задачи', 'NOUN'), ('за', 'ADP'), ('конечное', 'ADJ'), ('время', 'NOUN'), ('.', 'PUNCT')], [('В', 'ADP'), ('старой', 'ADJ'), ('трактовке', 'NOUN'), ('вместо', 'ADP'), ('слова', 'NOUN'), ('"', 'PUNCT'), ('порядок', 'NOUN'), ('"', 'PUNCT'), ('использовалось', 'VERB'), ('слово', 'NOUN'), ('"', 'PUNCT'), ('последовательность', 'NOUN'), ('"', 'PUNCT'), (',', 'PUNCT'), ('но', 'CCONJ'), ('по', 'ADP'), ('мере', 'NOUN'), ('развития', 'NOUN'), ('параллельности', 'NOUN'), ('в', 'ADP'), ('работе', 'NOUN'), ('компьютеров', 'NOUN'), ('слово', 'NOUN'), ('"', 'PUNCT'), (

In [ ]:
test_sent = conver_to_test_sent(full_test)[0]
print(test_sent)

['Алгоритм', ',', 'от', 'имени', 'учёного', 'аль', '-', 'Хорезми', ',', '-', 'точный', 'набор', 'инструкций', ',', 'описывающих', 'порядок', 'действий', 'исполнителя', 'для', 'достижения', 'результата', 'решения', 'задачи', 'за', 'конечное', 'время', '.']


### 1.Проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации

In [ ]:
unigram_tagger = UnigramTagger(train_data)
display(unigram_tagger.tag(test_sent), unigram_tagger.evaluate(test_data))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.8772537323492737

In [ ]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
display(bigram_tagger.tag(test_sent), bigram_tagger.evaluate(test_data))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.8829828463586425

In [ ]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
display(trigram_tagger.tag(test_sent), trigram_tagger.evaluate(test_data))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.882081353418933

In [ ]:
# Комбинация тэггеров
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test_data)

0.8814747413473528

### 2.Написать свой теггер как на занятии, но улучшить попробовать разные векторайзеры, добавить знание не только букв и слов но и совместно объединить эти признаки

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form.lower(), token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form.lower(), token.upos) for token in sent])
    
fdata_sent_train = []
for sent in full_train[:]:
    fdata_sent_train.append([token.form.lower() for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form.lower() for token in sent])
    
    
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [ ]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label) 
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [ ]:
# берем разные векторайзеры, анализируем по буквам
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(871526, 149809)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.943644053516665
(871526, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.9471826239342163
(871526, 149809)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.9487749806221144


In [ ]:
# берем разные векторайзеры, анализируем по словам
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='word')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(871526, 99485)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.7531847133757962
(871526, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.7722255922892866
(871526, 99485)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.7532605398847437


In [ ]:
# В случае совмещения двух признаков (буквы, слова) код успешно отработал только при первом запуске
# При последующих запусках выдавал ошибку. 
# Для ликвидации ошибки делалось уменьшение n_jobs. 
# После запуска в другой день и установки  n_jobs=4 ошибка не выдалась.
import numpy as np
from scipy.sparse import hstack

# берем векторайзер TfidfVectorizer по буквам и HashingVectorizer по словам
scaler = StandardScaler(with_mean=False)
coder_1 = TfidfVectorizer(ngram_range=(1, 5), analyzer='char')
coder_2 = HashingVectorizer(ngram_range=(1, 5), analyzer='word')

X_train_1 = coder_1.fit_transform(train_tok)
X_test_1 = coder_1.transform(test_tok)

X_train_2 = coder_2.fit_transform(train_tok)
X_test_2 = coder_2.transform(test_tok)


X_train = hstack((X_train_1,X_train_2))
X_test = hstack((X_test_1,X_test_2))

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)    


print(X_train.shape)
lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=4)
lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)

print('TfidfVectorizer_char + HashingVectorizer_word :', accuracy_score(test_enc_labels, pred))

(871526, 1198385)
TfidfVectorizer_char + HashingVectorizer_word : 0.9441327132409935


#### сравнить все реализованные методы сделать выводы

* 0.88147 - скор по комбинации из UnigramTagger BigramTagger TrigramTagger при методе nltk.tag
* 0.94877 - скор LogisticRegression в сочетании с TfidfVectorizer по буквам при методе Vectorizer






## Задание 2

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы (с доп информацией без) так же с учётом соседей и без них
4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера не забудьте разделить выборку

In [2]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.8 MB/s 


In [3]:
import corus

In [1]:

!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-12-05 22:28:15--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   946KB/s    in 2.0s    

2022-12-05 22:28:18 (946 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [4]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [5]:
!rm collection5.zip

In [6]:
!ls

Collection5  sample_data


In [7]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
rec = next(records)

In [8]:
rec.spans

[Ne5Span(
     index='T1',
     type='PER',
     start=0,
     stop=8,
     text='Медведев'
 ), Ne5Span(
     index='T2',
     type='ORG',
     start=55,
     stop=58,
     text='ПВО'
 ), Ne5Span(
     index='T3',
     type='GEOPOLIT',
     start=72,
     stop=74,
     text='РФ'
 ), Ne5Span(
     index='T4',
     type='PER',
     start=75,
     stop=91,
     text='Дмитрий Медведев'
 ), Ne5Span(
     index='T5',
     type='ORG',
     start=133,
     stop=150,
     text='Вооруженных силах'
 ), Ne5Span(
     index='T6',
     type='ORG',
     start=235,
     stop=238,
     text='ПВО'
 ), Ne5Span(
     index='T7',
     type='LOC',
     start=239,
     stop=266,
     text='Московского военного округа'
 ), Ne5Span(
     index='T8',
     type='MEDIA',
     start=288,
     stop=297,
     text='Интерфакс'
 ), Ne5Span(
     index='T9',
     type='LOC',
     start=323,
     stop=350,
     text='Московского военного округа'
 ), Ne5Span(
     index='T10',
     type='ORG',
     start=377,
     stop=3

In [9]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from razdel import tokenize

In [11]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        is_person = False
        for person in rec.spans:
            if (token.start >= person.start) and (token.stop <= person.stop):
                is_person = True
                break
        words.append([token.text, 'PERSON' if is_person else 'NO_PERSON'])
    words_docs.extend(words)

In [12]:
import pandas as pd

In [13]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [14]:
df_words['tag'].value_counts()

NO_PERSON    219069
PERSON        46197
Name: tag, dtype: int64

In [15]:
df_words.head(20)

,word,tag
0,С,PERSON
1,.,PERSON
2,Собянин,PERSON
3,провел,NO_PERSON
4,ряд,NO_PERSON
5,перестановок,NO_PERSON
6,в,NO_PERSON
7,московском,NO_PERSON
8,правительстве,NO_PERSON
9,Мэр,NO_PERSON


### 1.взять нер из nltk

In [16]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk

In [17]:
nltk.download('maxent_ne_chunker')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [18]:
nltk.pos_tag(nltk.word_tokenize(rec.text))

[('Новым', 'JJ'),
 ('президентом', 'NNP'),
 ('Перу', 'NNP'),
 ('становится', 'NNP'),
 ('националист', 'NNP'),
 ('О.Умала', 'NNP'),
 ('В', 'NNP'),
 ('Перу', 'NNP'),
 ('на', 'NNP'),
 ('выборах', 'NNP'),
 ('президента', 'NNP'),
 ('побеждает', 'NNP'),
 ('левый', 'NNP'),
 ('националист', 'NNP'),
 ('Ольянта', 'NNP'),
 ('Умала', 'NNP'),
 ('.', '.'),
 ('48-летний', 'JJ'),
 ('бывший', 'JJ'),
 ('кадровый', 'NN'),
 ('военный', 'NNP'),
 ('набирает', 'NNP'),
 ('более', 'NNP'),
 ('половины', 'NNP'),
 ('отданных', 'NNP'),
 ('электоратом', 'NNP'),
 ('голосов', 'NNP'),
 (',', ','),
 ('передает', 'NNP'),
 ('телеканал', 'NNP'),
 ('Al', 'NNP'),
 ('Jazeera', 'NNP'),
 ('со', 'NNP'),
 ('ссылкой', 'NNP'),
 ('на', 'NNP'),
 ('данные', 'NNP'),
 ('exit-polls', 'NNS'),
 ('.', '.'),
 ('Этот', 'VB'),
 ('кандидат', 'JJ'),
 ('набрал', 'JJ'),
 ('52,6', 'CD'),
 ('%', 'NN'),
 ('голосов', 'JJ'),
 ('опрошенных', 'NNP'),
 (',', ','),
 ('тогда', 'NNP'),
 ('как', 'NNP'),
 ('представительница', 'NNP'),
 ('правой', 'NNP'),
 ('к

In [19]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(rec.text))) if hasattr(chunk, 'label') }

{('Кейко Фухимори', 'PERSON'),
 ('Новым', 'PERSON'),
 ('Ольянта Умала', 'PERSON'),
 ('Считается', 'PERSON'),
 ('Эквадора', 'PERSON')}

#### Вывод по nltk
NER из nltk классифицирует с ошибками именованные сущности в тексте на русском языке.

### 3.написать свой нер, попробовать разные подходы (с доп информацией и без), также с учётом соседей и без них

In [20]:
records = load_ne5(dir)
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
       
        result = 'None'        
        
        for item in rec.spans:            
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'PER'):
                result = 'PER'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'ORG'):
                result = 'ORG'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'MEDIA'):
                result = 'MEDIA'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'LOC'):
                result = 'LOC'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'GEOPOLIT'):
                result = 'GEOPOLIT'
                break
                
    
        words.append([token.text, result])
    words_docs.extend(words)

In [21]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [22]:
df_words['tag'].value_counts()

None        219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [23]:
df_words.head(20)

,word,tag
0,Медведев,PER
1,уволил,None
2,трех,None
3,генералов,None
4,и,None
5,начальника,None
6,московской,None
7,ПВО,ORG
8,Президент,None
9,РФ,GEOPOLIT


In [24]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [25]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [26]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [27]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [28]:
def custom_standardization(input_data):
        return input_data

def data_prep(train_data, seq_len=1, vocab_size = 30000):    
    
    vocab_size = 30000
    #seq_len = 1

    vectorize_layer = TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=seq_len)


    # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
    text_data = train_data.map(lambda x, y: x)
    vectorize_layer.adapt(text_data)
    return vectorize_layer

In [29]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gPool = tf.keras.layers.GlobalMaxPooling1D()
        self.fc1 = tf.keras.layers.Dense(300, activation='relu')
        self.fc2 = tf.keras.layers.Dense(50, activation='relu')
        self.fc3 = tf.keras.layers.Dense(len(df_words['tag'].value_counts()), activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        return self.fc3(concat_x)

In [30]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 1, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 73s 6ms/step - loss: 0.2840 - accuracy: 0.9178 - val_loss: 0.2078 - val_accuracy: 0.9389
Epoch 2/5
12444/12444 [==============================] - 56s 5ms/step - loss: 0.1208 - accuracy: 0.9638 - val_loss: 0.2084 - val_accuracy: 0.9417
Epoch 3/5
12444/12444 [==============================] - 58s 5ms/step - loss: 0.1079 - accuracy: 0.9656 - val_loss: 0.2194 - val_accuracy: 0.9416
Epoch 4/5
12444/12444 [==============================] - 56s 4ms/step - loss: 0.1034 - accuracy: 0.9663 - val_loss: 0.2226 - val_accuracy: 0.9409
Epoch 5/5
12444/12444 [==============================] - 56s 4ms/step - loss: 0.1009 - accuracy: 0.9668 - val_loss: 0.2335 - val_accuracy: 0.9404


In [32]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 3, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 60s 5ms/step - loss: 0.2932 - accuracy: 0.9143 - val_loss: 0.2083 - val_accuracy: 0.9378
Epoch 2/5
12444/12444 [==============================] - 57s 5ms/step - loss: 0.1254 - accuracy: 0.9626 - val_loss: 0.2041 - val_accuracy: 0.9385
Epoch 3/5
12444/12444 [==============================] - 56s 5ms/step - loss: 0.1096 - accuracy: 0.9653 - val_loss: 0.2449 - val_accuracy: 0.9413
Epoch 4/5
12444/12444 [==============================] - 57s 5ms/step - loss: 0.1042 - accuracy: 0.9663 - val_loss: 0.2838 - val_accuracy: 0.9408
Epoch 5/5
12444/12444 [==============================] - 56s 4ms/step - loss: 0.1015 - accuracy: 0.9667 - val_loss: 0.3205 - val_accuracy: 0.8836


In [33]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 5, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 56s 4ms/step - loss: 0.2958 - accuracy: 0.9135 - val_loss: 0.2124 - val_accuracy: 0.9357
Epoch 2/5
12444/12444 [==============================] - 56s 5ms/step - loss: 0.1260 - accuracy: 0.9623 - val_loss: 0.2341 - val_accuracy: 0.9400
Epoch 3/5
12444/12444 [==============================] - 57s 5ms/step - loss: 0.1093 - accuracy: 0.9653 - val_loss: 0.2604 - val_accuracy: 0.9413
Epoch 4/5
12444/12444 [==============================] - 56s 5ms/step - loss: 0.1039 - accuracy: 0.9662 - val_loss: 0.2826 - val_accuracy: 0.9411
Epoch 5/5
12444/12444 [==============================] - 56s 4ms/step - loss: 0.1008 - accuracy: 0.9668 - val_loss: 0.3053 - val_accuracy: 0.8808


#### Вывод по modelNER
В собственном нер подходы с учетом соседей и без мало различаются по результатам.

### 2.проверить deeppavlov

In [34]:
# установка deeppavlov

!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Successfully uninstalled tensorflow-2.9.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 26.7 MB/s 
     |████████████████████████████████| 5.8 MB 67.5 MB/s 
     |████████████████████████████████| 182 kB 57.1 MB/s 
     |████████████████████████████████| 7.6 MB 53.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 22.3 MB/s 
     |████████████████████████████████| 57 kB 4.7 MB/s 
     |████████████████████████████████| 145 kB 71.9 MB/s 
     |████████████████████████████████| 59 kB 8.2 MB/s 
     |████████████████████████████████| 54 kB 3.9 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 55 kB 4.9 MB/s 
     |████████████████████████████████| 8.2 MB 69.6 M

In [35]:
!python -m deeppavlov install squad_bert
!python -m deeppavlov install ner_ontonotes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 23.8 MB/s 
     |████████████████████████████████| 6.6 MB 39.3 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.25.1
    Uninstalling transformers-4.25.1:
      Successfully uninstalled transformers-4.25.1
2022-12-05 22:48:46.966 WARNING in 'deeppavlov.core.common.file'['file'] at line 39: 

################################################################################
# The model 'ner_ontonotes' has been removed from the DeepPavlov configs.
# The model 'ner_ontonotes_bert' is used instead.
# To disable this message please switch to '

In [36]:
import deeppavlov
from deeppavlov import configs, build_model

In [37]:
!python -m deeppavlov install ner_ontonotes_bert_mult

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
ner_dp_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
result_1 = ner_dp_model([rus_document])

for i in range(len(result_1[0][0])):
     if result_1 [1][0][i] != 'O':
         print(result_1[0][0][i], result_1[1][0][i])

2022-12-05 22:49:26.219 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_mult_torch_crf.tar.gz to /root/.deeppavlov/models/ner_ontonotes_bert_mult_torch_crf.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_mult_torch_crf.tar.gz to /root/.deeppavlov/models/ner_ontonotes_bert_mult_torch_crf.tar.gz
100%|██████████| 1.39G/1.39G [01:35<00:00, 14.5MB/s]
2022-12-05 22:51:03.66 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting /root/.deeppavlov/models/ner_ontonotes_bert_mult_torch_crf.tar.gz archive into /root/.deeppavlov/models/ner_ontonotes_torch_bert_mult_crf
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/models/ner_ontonotes_bert_mult_torch_crf.tar.gz archive into /root/.deeppavlov/models/ner_ontonotes_torch_bert_mult_crf


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

Нью B-GPE
- I-GPE
Йорк I-GPE
США B-GPE
30 B-DATE
апреля I-DATE
2020 I-DATE
01 B-TIME
01 B-TIME
США B-GPE
Дональда B-PERSON
Трампа I-PERSON
100 B-MONEY
млн I-MONEY
доз I-MONEY
конца B-DATE
2020 I-DATE
года I-DATE
Bloomberg B-ORG


/usr/local/lib/python3.8/dist-packages/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  ../aten/src/ATen/native/TensorCompare.cpp:402.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


#### Вывод по deeppavlov
В DeepPavlov мультиязычный NER версии ner_ontonotes_bert_mult отлично классифицирует слова русского текста по типу именованных сущностей, к которым они принадлежат (имена собственные, географические названия, даты, валюты и другие).

### 4.Дополнительно: NER в STANZA

In [39]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 35.0 MB/s 
     |████████████████████████████████| 240 kB 74.2 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=e38aa8c69722a0538340d8dbcb3c5d1ae07b790052318097c64e16f729cdc097
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [40]:
import stanza
stanza.download('ru')
def stanza_nlp_ru(text):
  nlp = stanza.Pipeline(lang='ru', processors='tokenize,ner')
  doc = nlp(text)
  print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')
stanza_nlp_ru(rus_document)

INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize


RuntimeError: ignored

#### Вывод по STANZA
Модель с NER в STANZA отработала быстрее, чем NER в DeepPavlov. Результат для текста на русском языке - отличный.